In [8]:
import pandas as pd
import numpy as np
import re

def clean_berlin_marathon_data(input_parquet_path, output_parquet_path):
    """
    Performs deep cleaning on the Berlin Marathon dataset.
    Standardizes column names, handles time conversions, harmonizes categorical data (gender),
    and removes physiologically impossible values based on official race limits.
    """
    
    print(f"--- Loading data from {input_parquet_path} ---")
    df = pd.read_parquet(input_parquet_path)
    
    # ---------------------------------------------------------
    # 1. COLUMN NORMALIZATION
    # ---------------------------------------------------------
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    
    col_mapping = {
        'sex': 'gender',
        'sex_place': 'gender_rank',
        'place_sex': 'gender_rank',
        'ac': 'age_group',
        'ak': 'age_group',
        'age_category': 'age_group',
        'division': 'age_group',
        'net': 'net_time',
        'netto': 'net_time',
        'chip_time': 'net_time',
        'final': 'net_time',       
        'gross': 'gross_time',
        'brutto': 'gross_time',
        'gun_time': 'gross_time',
        'nation': 'country',
        'nat': 'country'
    }
    df.rename(columns=col_mapping, inplace=True)
    
    print(f"Columns after normalization: {df.columns.tolist()}")

    # ---------------------------------------------------------
    # 2. DATA TYPE CONVERSION & PARSING
    # ---------------------------------------------------------
    
    def parse_time_string(time_str):
        # Convert to string to handle potential categorical objects safely
        s = str(time_str).strip()
        if pd.isna(time_str) or s == '-' or s == '' or s.lower() == 'nan':
            return np.nan
        try:
            parts = s.split(':')
            if len(parts) == 3: # H:M:S
                return int(parts[0])*3600 + int(parts[1])*60 + float(parts[2])
            elif len(parts) == 2: # M:S
                return int(parts[0])*60 + float(parts[1])
            else:
                return np.nan
        except:
            return np.nan

    print("Processing 'net_time'... converting to seconds.")
    
    if 'net_time' in df.columns:
        # Apply parsing
        # We assume net_time might be categorical, so we ensure it's treated as string inside the helper
        df['time_seconds'] = df['net_time'].apply(parse_time_string)
        
        # --- FIX IS HERE: FORCE NUMERIC TYPE ---
        # This converts any 'category' or 'object' dtype strictly to float64
        # 'coerce' turns any parsing errors into NaNs automatically
        df['time_seconds'] = pd.to_numeric(df['time_seconds'], errors='coerce')
        
        # Create a readable timedelta column
        df['finish_time'] = pd.to_timedelta(df['time_seconds'], unit='s')
    else:
        print(f"CRITICAL ERROR: 'net_time' still not found. Available columns: {df.columns.tolist()}")
        return None

    # ---------------------------------------------------------
    # 3. CATEGORICAL STANDARDIZATION (Gender)
    # ---------------------------------------------------------
    if 'gender' in df.columns:
        print("Standardizing gender values...")
        gender_map = {
            'm': 'M', 'män': 'M', 'men': 'M', 'h': 'M',
            'w': 'F', 'f': 'F', 'fra': 'F', 'women': 'F', 'd': 'F'
        }
        df['gender'] = df['gender'].astype(str).str.lower().str[:3].map(gender_map).fillna('Unknown')

    # ---------------------------------------------------------
    # 4. PHYSIOLOGICAL & INTEGRITY FILTERS
    # ---------------------------------------------------------
    print("Applying physiological filters (Data Integrity)...")
    initial_rows = len(df)
    
    # A. Remove rows with no valid time (NaNs created by the numeric coercion)
    df = df.dropna(subset=['time_seconds'])
    
    # B. World Record Lower Bound Check (approx 1h59)
    min_seconds = 1 * 3600 + 59 * 60 
    
    # C. Upper Bound Check (Official Limit 6h15m)
    max_seconds = 6 * 3600 + 15 * 60 
    
    # Now this comparison works because 'time_seconds' is guaranteed to be float64
    df_clean = df[(df['time_seconds'] >= min_seconds) & (df['time_seconds'] <= max_seconds)].copy()
    
    dropped_rows = initial_rows - len(df_clean)
    print(f"Dropped {dropped_rows} rows ({dropped_rows/initial_rows:.2%}) due to invalid times or outliers (> 6h15m).")
    
    # Ensure year is numeric
    if 'year' in df_clean.columns:
        df_clean['year'] = pd.to_numeric(df_clean['year'], errors='coerce')

    # ---------------------------------------------------------
    # 5. SAVE CLEANED DATA
    # ---------------------------------------------------------
    print(f"Saving cleaned dataset to {output_parquet_path}...")
    df_clean.to_parquet(output_parquet_path, engine='pyarrow')
    print("Cleaning complete. Ready for analysis.")
    
    return df_clean

# --- EXECUTION BLOCK ---
input_file = "Dataset_Berlin_Marathon_1999-2025.parquet"
output_file = "Dataset_Berlin_Cleaned_Analysis_Ready.parquet"

df_cleaned = clean_berlin_marathon_data(input_file, output_file)

if df_cleaned is not None:
    print("\nSample of Cleaned Data:")
    print(df_cleaned[['year', 'gender', 'net_time', 'time_seconds']].head())

--- Loading data from Dataset_Berlin_Marathon_1999-2025.parquet ---
Columns after normalization: ['year', 'gender', 'name', 'country', 'starting_num', 'age_group', '5km', '10km', '15km', '20km', 'half', '25km', '30km', '35km', '40km', 'net_time']
Processing 'net_time'... converting to seconds.
Standardizing gender values...
Applying physiological filters (Data Integrity)...
Dropped 7445 rows (0.85%) due to invalid times or outliers (> 6h15m).
Saving cleaned dataset to Dataset_Berlin_Cleaned_Analysis_Ready.parquet...
Cleaning complete. Ready for analysis.

Sample of Cleaned Data:
   year gender  net_time  time_seconds
0  1999      M  02:06:44        7604.0
1  1999      M  02:06:57        7617.0
2  1999      M  02:08:31        7711.0
3  1999      M  02:09:56        7796.0
4  1999      M  02:10:37        7837.0
